In [3]:
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

import scipy.sparse as sp
import pandas as pd
from tqdm import tqdm
from pathlib import Path

In [4]:
discard_dataset_path = "D://Mahjong group project data/archive/discard_datasets"
dataset_folder = Path(discard_dataset_path) / "2019"      #path to file folder

In [5]:
file = list(dataset_folder.iterdir())[0]       #path to single file

game = sp.load_npz(file)     #loading file

matrix = sp.vstack(game) # Stacking them to get Nx511

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Mahjong group project data\\archive\\discard_datasets\\2019'

In [ ]:
array = game.toarray()  # Converting from scipy sparse matrix to numpy array

array.shape 

In [ ]:
X = array[:, :510]           # Board State
y = array[:, -1:].flatten()  # Discarded Tile

X.shape